## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime, date

## Download

In [6]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"),)
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"d","Entity":"demo_region_count","Type":0},{"Name":"r","Entity":"region_lastmonth","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"mregion"},"Name":"demo_region_count.mregion"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"case_count"}},"Function":0},"Name":"Sum(demo_region_count.case_count)"},{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"zip2"},"Name":"demo_region_count.zip2"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"case_count"}},"Function":0},"Name":"Sum(region_lastmonth.case_count)"}],"Where":[{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"zip2"}}],"Values":[[{"Literal":{"Value":"null"}}]]}}}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"mregion"}}],"Values":[[{"Literal":{"Value":"\'Unknown\'"}}]]}}}}}],"OrderBy":[{"Direction":2,"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"case_count"}},"Function":0}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1},"ExecutionMetricsKind":1}}]},"QueryId":"","ApplicationContext":{"DatasetId":"2e66a584-6b5f-4c9f-bffd-12d35d2b8442","Sources":[{"ReportId":"40085a43-f04a-4a8b-ae72-c38ca32531a5","VisualId":"136278efd4c6f694b9b7"}]}}],"cancelQueries":[],"modelId":358821}'

In [7]:
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,da;q=0.8",
    "activityid": "07a2f3cb-ea39-4519-8ee0-b5605aa78145",
    "content-type": "application/json;charset=UTF-8",
    "requestid": "a5565ffd-b522-00a7-afe8-5c4f3a3ce843",
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-powerbi-resourcekey": "69a5588e-f8a1-4aef-aac5-ca27ee543007",
    "Referer": "https://app.powerbigov.us/",
    "Referrer-Policy": "strict-origin-when-cross-origin",
}
json_data = json.loads(REQUEST_BODY)

In [8]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/home/runner/.local/share/virtualenvs/california-coronavirus-scrapers-dxuBXRsm/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
data = response.json()

## Parse

In [10]:
timestamp = response.headers["Date"]

In [11]:
timestamp

'Sun, 19 Jun 2022 08:19:41 GMT'

In [12]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [13]:
zip_codes = ds["ValueDicts"]["D1"]

In [14]:
rows = ds["PH"][0]["DM0"]

In [15]:
rows

[{'S': [{'N': 'G0', 'T': 1, 'DN': 'D0'},
   {'N': 'G1', 'T': 1, 'DN': 'D1'},
   {'N': 'M0', 'T': 4},
   {'N': 'M1', 'T': 4}],
  'C': [0, 0, 14323, 191]},
 {'C': [1, 14249, 338], 'R': 1},
 {'C': [1, 2, 9219, 148]},
 {'C': [2, 3, 6290, 287]},
 {'C': [0, 4, 5715, 175]},
 {'C': [1, 5, 4412, 47]},
 {'C': [2, 6, 4019, 228]},
 {'C': [3, 7, 3724, 133]},
 {'C': [1, 8, 30], 'R': 4},
 {'C': [2, 9, 3349, 229]},
 {'C': [3, 10, 3172, 73]},
 {'C': [11, 2663, 53], 'R': 1},
 {'C': [1, 12, 2050, 26]},
 {'C': [0, 13, 1504, 76]},
 {'C': [2, 14, 1376, 123]},
 {'C': [15, 796, 54], 'R': 1},
 {'C': [16, 483, 24], 'R': 1},
 {'C': [3, 17, 403, 18]},
 {'C': [2, 18, 335, 45]},
 {'C': [1, 19, 322, 1]},
 {'C': [3, 20, 273, 9]},
 {'C': [0, 21, 257, 8]},
 {'C': [2, 22, 183, 35]},
 {'C': [23, 175, 17], 'R': 1},
 {'C': [1, 24, 5], 'R': 4},
 {'C': [0, 25, 144, 6]},
 {'C': [3, 26, 130, 4]},
 {'C': [1, 27, 113, 6]},
 {'C': [2, 28, 107, 3]},
 {'C': [1, 29, 104, 1]},
 {'C': [0, 30, 89, 3]},
 {'C': [1, 31, 86], 'R': 8},
 {'C

In [16]:
confirmed_cases = []

cases = 0

for row in rows:
    # in some cases, data are encoded in separate keys
    keys = row.keys()
    # pull out the list containing values for most rows
    cells = row["C"]
    # second value in the list is expected to be an iterator/index
    index = cells[1]
    # if this key is present, the value for this place is identical to the previous row
    if "R" in keys:
        if row["R"] == 1:
            cases = cells[1]
            index = cells[0]
        else:
            cases = cells[2]
            index = cells[1]
    # every other row has its value in the third position of the list
    else:
        cases = cells[2]
    # append values to master list
    confirmed_cases.append(
        (
            index,
            cases,
        )
    )

In [17]:
matched = [(zip_codes[index], cases) for index, cases in confirmed_cases]

In [18]:
df = pd.DataFrame(matched, columns=["zip_code", "confirmed_cases"])

In [19]:
df

,zip_code,confirmed_cases
0,93905,14323
1,93906,14249
2,93960,9219
3,93955,6290
4,93901,5715
5,93927,4412
6,93933,4019
7,93907,3724
8,93930,30
9,93940,3349


Match up place names with zip codes

In [20]:
df = df.loc[~df.zip_code.str.contains("Other")]

In [21]:
zip_code_names = {
    "93902": "Prunedale",
    "93907": "Salinas",
    "95004": "Aromas",
    "95012": "Castroville",
    "95039": "Moss Landing",
    "95076": "Watsonville",
    "93920": "Big Sur",
    "93921": "Carmel",
    "93923": "Carmel",
    "93924": "Carmel Valley",
    "93933": "Marina",
    "93940": "Monterey",
    "93942": "Monterey",
    "93950": "Pacific Grove",
    "93953": "Pebble Beach",
    "93955": "Seaside",
    "93901": "Salinas",
    "93905": "Salinas",
    "93906": "Salinas",
    "93908": "Salinas",
    "93912": "Boronda",
    "93915": "Salinas",
    "93962": "Spreckels",
    "93450": "San Ardo",
    "93925": "Chualar",
    "93926": "Gonzales",
    "93927": "Greenfield",
    "93930": "King City",
    "93954": "San Lucas",
    "93960": "Soledad",
    "93922": "Carmel",
    "93426": "Bradley",
    "93451": "San Miguel",
}

In [22]:
df["area_name"] = df.zip_code.map(zip_code_names)

/tmp/ipykernel_2255/640265091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["area_name"] = df.zip_code.map(zip_code_names)


In [23]:
df["area_name"] = df["zip_code"] + ": " + df["area_name"]

/tmp/ipykernel_2255/3669557735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["area_name"] = df["zip_code"] + ": " + df["area_name"]


In [24]:
df = df[(df.zip_code != "OtherPBS")]

Set column with county date

In [25]:
df["county_date"] = pd.to_datetime(timestamp)

In [26]:
df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")

In [27]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

In [28]:
df.insert(0, "county", "Monterey")

Clean up for export

In [29]:
export_df = df[
    ["county", "area_name", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"area_name": "area", "zip_code": "zip"})

## Vet

In [30]:
try:
    assert not len(export_df) < 33
except AssertionError:
    raise AssertionError("Monterey County's zip code scraper is missing rows")

In [31]:
try:
    assert not len(export_df) > 33
except AssertionError:
    raise AssertionError(
        f"Monterey County's zip code scraper has {len(export_df)-29} more rows than before"
    )

In [32]:
export_df[~export_df.area.isin(old.area)]

NameError: name 'old' is not defined

In [56]:
export_df

,county,area,confirmed_cases,county_date,zip
0,Monterey,93905: Salinas,14323,2022-06-16,93905
1,Monterey,93906: Salinas,14249,2022-06-16,93906
2,Monterey,93960: Soledad,9219,2022-06-16,93960
3,Monterey,93955: Seaside,6290,2022-06-16,93955
4,Monterey,93901: Salinas,5715,2022-06-16,93901
5,Monterey,93927: Greenfield,4412,2022-06-16,93927
6,Monterey,93933: Marina,4019,2022-06-16,93933
7,Monterey,93907: Salinas,3724,2022-06-16,93907
8,Monterey,93930: King City,30,2022-06-16,93930
9,Monterey,93940: Monterey,3349,2022-06-16,93940


## Export

Set date

In [57]:
tz = pytz.timezone("America/Los_Angeles")

In [58]:
today = datetime.now(tz).date()

In [59]:
slug = "monterey"

In [60]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [61]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [62]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [63]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [64]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)